In [ ]:
!pip install earthengine-api

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-harshitpmd2003')

In [3]:
# Define the AOI: Specific districts in India
districts = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.Or(
        ee.Filter.eq('ADM2_NAME', 'Agra'),
        ee.Filter.eq('ADM2_NAME', 'Mathura'),
        ee.Filter.eq('ADM2_NAME', 'Bharatpur')
    ))

In [7]:
import geemap

In [8]:
# Filter for the Agra district
agra = districts.filter(ee.Filter.eq('ADM2_NAME', 'Agra'))
agra_centroid = agra.geometry().centroid()
agra_centroid_coordinates = agra_centroid.coordinates().getInfo()
print(f"Centroid coordinates of Agra: {agra_centroid_coordinates}")

Map_Agra = geemap.Map(center=agra_centroid_coordinates[::-1], zoom=10)
Map_Agra.addLayer(agra, {}, 'Agra')
Map_Agra

Centroid coordinates of Agra: [78.06325784158271, 27.007783339668872]


Map(center=[27.007783339668872, 78.06325784158271], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# Filter for the Bharatpur district
bharatpur = districts.filter(ee.Filter.eq('ADM2_NAME', 'Bharatpur'))
bharatpur_centroid = bharatpur.geometry().centroid()
bharatpur_centroid_coordinates = bharatpur_centroid.coordinates().getInfo()
print(f"Centroid coordinates of Bharatpur: {bharatpur_centroid_coordinates}")

Map_Bharatpur = geemap.Map(center=bharatpur_centroid_coordinates[::-1], zoom=9)
Map_Bharatpur.addLayer(bharatpur, {}, 'Bharatpur')
Map_Bharatpur

Centroid coordinates of Bharatpur: [77.27104931472743, 27.250639927444407]


Map(center=[27.250639927444407, 77.27104931472743], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
# Filter for the Marthura district
mathura = districts.filter(ee.Filter.eq('ADM2_NAME', 'Mathura'))

mathura_centroid = mathura.geometry().centroid()
mathura_centroid_coordinates = mathura_centroid.coordinates().getInfo()
print(f"Centroid coordinates of Mathura: {mathura_centroid_coordinates}")

Map_Mathura = geemap.Map(center=mathura_centroid_coordinates[::-1], zoom=10)
Map_Mathura.addLayer(mathura, {}, 'Marthura')
Map_Mathura

Centroid coordinates of Mathura: [77.62098339697215, 27.600403055488048]


Map(center=[27.600403055488048, 77.62098339697215], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
# Combine the geometries of Agra, Bharatpur, and Mathura
combined_geometry = districts.geometry()

# Get the centroid of the combined geometry to use as the map center
centroid = combined_geometry.centroid().coordinates().getInfo()
center_coordinates = [centroid[1], centroid[0]]  # [latitude, longitude]

Map_combined = geemap.Map(center=center_coordinates, zoom=9)
Map_combined.addLayer(combined_geometry, {}, 'ABM')
Map_combined

Map(center=[27.278104285773832, 77.60387814258341], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
# Union the geometries of Agra, Bharatpur, and Mathura
union_geometry = districts.geometry().dissolve()

# Get the centroid of the combined geometry to use as the map center
centroid = union_geometry.centroid().coordinates().getInfo()
center_coordinates = [centroid[1], centroid[0]]  # [latitude, longitude]

Map_union = geemap.Map(center=center_coordinates, zoom=9)
Map_union.addLayer(union_geometry, {}, 'Selected Districts')
Map_union

Map(center=[27.27810428577392, 77.60387814258337], controls=(WidgetControl(options=['position', 'transparent_b…

# sentinel-1 image collection

In [13]:
# Date range
startDate = '2023-01-01'
endDate = '2023-12-31'

In [16]:
# Load Sentinel-1 Image Collections
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(union_geometry) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV') \
    .median() \
    .clip(union_geometry)

In [17]:
# Visualization parameters
sentinel1_visParams = {
    'bands': ['VV'],
    'min': -30,  # -35
    'max': 10,   # 5
}

# Create a map for Sentinel-1 imagery
Map_sentinel1 = geemap.Map(center=center_coordinates, zoom=9)
Map_sentinel1.addLayer(sentinel1, sentinel1_visParams, 'Original Sentinel-1')
Map_sentinel1.centerObject(union_geometry, zoom=9)
Map_sentinel1

Map(center=[27.27810428577392, 77.60387814258337], controls=(WidgetControl(options=['position', 'transparent_b…

# Sentinel-2 image collection

In [19]:
sentinel2_with_cloud_mask= ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(union_geometry) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B8', 'B4', 'B3']) \
    .median() \
    .clip(union_geometry)

In [20]:
# Visualization parameters
sentinel2_visParams = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 5000,
}

# Create a map for Sentinel-2 imagery
Map_sentinel2_with_cloud_mask = geemap.Map(center=center_coordinates, zoom=9)
Map_sentinel2_with_cloud_mask.addLayer(sentinel2_with_cloud_mask, sentinel2_visParams,'Sentinel-2 Image')
Map_sentinel2_with_cloud_mask.centerObject(union_geometry, zoom=9)
Map_sentinel2_with_cloud_mask

Map(center=[27.27810428577392, 77.60387814258337], controls=(WidgetControl(options=['position', 'transparent_b…

# GAN

In [45]:
import numpy as np
import requests
import tensorflow as tf
import io

def get_sentinel_data(image, size):
    url = image.getThumbURL({
        'dimensions': size,
        'format': 'NPY'
    })
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    image_data = np.load(io.BytesIO(response.content))
    return image_data

size = [256, 256]  # Example size

s1_images = get_sentinel_data(sentinel1, size)
s2_images = get_sentinel_data(sentinel2_with_cloud_mask, size)

# Handle multi-band Sentinel-2 images and convert to float32
s2_images_b8 = s2_images['B8'].astype(np.float32)
s2_images_b4 = s2_images['B4'].astype(np.float32)
s2_images_b3 = s2_images['B3'].astype(np.float32)

# Stack the bands along the last axis
s2_images = np.stack([s2_images_b8, s2_images_b4, s2_images_b3], axis=-1)

# Ensure Sentinel-1 images are also float32
s1_images = s1_images.astype(np.float32)

# Expand dimensions to match expected input shape
s1_images = np.expand_dims(s1_images, axis=-1)


# GAN model design

In [46]:
from tensorflow.keras import layers

# Define input and output shapes
input_shape = (256, 256, 1)  # Example input shape for Sentinel-1 images
output_shape = (256, 256, 3)  # Example output shape for Sentinel-2 images

# Generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(3, (3, 3), padding='same', activation='tanh'))
    return model

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=output_shape))
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build and compile the models
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# GAN model
gan_input = layers.Input(shape=input_shape)
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')


# Training the GAN

In [47]:
# Training parameters
epochs = 10000
batch_size = 64

# Function to train the GAN
def train_gan(gan, generator, discriminator, s1_images, s2_images, epochs, batch_size):
    for epoch in range(epochs):
        # Select a random batch of Sentinel-1 and Sentinel-2 images
        idx = np.random.randint(0, s1_images.shape[0], batch_size)
        s1_batch = s1_images[idx]
        s2_batch = s2_images[idx]

        # Generate synthetic images
        generated_images = generator.predict(s1_batch)

        # Train the discriminator
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))
        d_loss_real = discriminator.train_on_batch(s2_batch, real_labels)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.train_on_batch(s1_batch, real_labels)

        # Print progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

# Train the GAN
train_gan(gan, generator, discriminator, s1_images, s2_images, epochs, batch_size)
generator.save('generator_model.h5')


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(64, 256, 3), dtype=float32). Expected shape (None, 256, 256, 3), but input has incompatible shape (64, 256, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(64, 256, 3), dtype=float32)
  • training=True
  • mask=None

#  Generate Fused Images

In [48]:
# Generate fused images using the trained generator
def generate_fused_images(generator, s1_images):
    fused_images = generator.predict(s1_images)
    return fused_images

# Generate fused images
fused_images = generate_fused_images(generator, s1_images)

# Save fused images to disk
for i, img in enumerate(fused_images):
    tf.keras.preprocessing.image.save_img(f"fused_image_{i}.png", img)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
